### ASF Code

In [ ]:
%%capture
import os # for chdir, getcwd, path.basename, path.exists

import pandas as pd # for DatetimeIndex
import gdal # for GetRasterBand, Open, ReadAsArray
import numpy as np #for log10, mean, percentile, power
import matplotlib.pylab as plb # for add_patch, add_subplot, figure, hist, imshow, set_title, xaxis,_label, text 
import matplotlib.pyplot as plt # for add_subplot, axis, figure, imshow, legend, plot, set_axis_off, set_data,
                                # set_title, set_xlabel, set_ylabel, set_ylim, subplots, title, twinx
import matplotlib.patches as patches  # for Rectangle
import matplotlib.animation as an # for FuncAnimation
from matplotlib import rc 

from asf_notebook import path_exists
from asf_notebook import asf_unzip
from asf_notebook import new_directory
from asf_notebook import jupytertheme_matplotlib_format

from IPython.display import HTML

In [ ]:
%%capture
jupytertheme_matplotlib_format()
%matplotlib inline 

In [ ]:
path = "/home/jovyan/notebooks/ASF/GEOS_657_Labs/2021/lab_4_data"
new_directory(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
time_series_path = 's3://asf-jupyter-data/time_series.zip'
time_series = os.path.basename(time_series_path)
!aws --region=us-east-1 --no-sign-request s3 cp $time_series_path $time_series

In [ ]:
if path_exists(time_series):
    asf_unzip(os.getcwd(), time_series)
    os.remove(time_series)

In [ ]:
datadirectory = '/home/jovyan/notebooks/ASF/GEOS_657_Labs/2021/lab_4_data/time_series/S32644X696260Y3052060sS1-EBD'
datefile = 'S32644X696260Y3052060sS1_D_vv_0092_mtfil.dates'
imagefile = 'S32644X696260Y3052060sS1_D_vv_0092_mtfil.vrt'
imagefile_cross = 'S32644X696260Y3052060sS1_D_vh_0092_mtfil.vrt'

In [ ]:
if path_exists(datadirectory):
    os.chdir(datadirectory)
print(f"current directory: {os.getcwd()}")

In [ ]:
#!ls *.vrt #Uncomment this line to see a List of the files 

In [ ]:
if path_exists(datefile):
    with open(datefile, 'r') as f:
        dates = f.readlines()
        tindex = pd.DatetimeIndex(dates)

In [ ]:
if path_exists(imagefile):
    j = 1
    print('Bands and dates for', imagefile)
    for i in tindex:
        print("{:4d} {}".format(j, i.date()),end=' ')
        j += 1
        if j%5 == 1: print()

In [ ]:
if path_exists(imagefile):
    img = gdal.Open(imagefile)

In [ ]:
print(img.RasterCount) # Number of Bands
print(img.RasterXSize) # Number of Pixels
print(img.RasterYSize) # Number of Lines

In [ ]:
band_num = 70 
band = img.GetRasterBand(band_num)

In [ ]:
raster = band.ReadAsArray()

In [ ]:
raster_sub = band.ReadAsArray(5, 20, 50, 50)

In [ ]:
raster_sub

In [ ]:
raster_1 = img.GetRasterBand(20).ReadAsArray()
raster_2 = img.GetRasterBand(27).ReadAsArray()

In [ ]:
plt.rcParams.update({'font.size': 12})
def show_image_histogram(raster, tindex, band_nbr, vmin=None, vmax=None, output_filename=None):
    assert 'plb' in globals(), 'Error: matplotlib.pylab must be imported as "plb"'
    assert 'plt' in globals(), 'Error: matplotlib.pyplot must be imported as "plt"'  
    
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    # plot image
    ax1.imshow(raster, cmap='gray', vmin=vmin, vmax=vmax)
    ax1.set_title('Image Band {} {}'.format(band_nbr, tindex[band_nbr-1].date()))
    vmin = np.percentile(raster, 2) if vmin==None else vmin
    vmax = np.percentile(raster, 98) if vmax==None else vmax
    ax1.xaxis.set_label_text('Linear stretch Min={} Max={}'.format(vmin, vmax))
    
    #plot histogram
    h = ax2.hist(raster.flatten(), bins=200, range=(0, 10000))
    ax2.grid()
    ax2.xaxis.set_label_text('Amplitude (Uncalibrated DN Values)')
    ax2.set_title('Histogram Band {} {}'.format(band_nbr, tindex[band_nbr-1].date()))
    
    if output_filename:
        plt.savefig(output_filename, dpi=300, transparent='true')

In [ ]:
show_image_histogram(raster_1, tindex, 20, vmin=2000, vmax=10000)

## SAR time series

In [ ]:
# Open the image and read the first raster band
band = img.GetRasterBand(1)

# Define the subset
subset = (400, 400, 600, 600)

In [ ]:
# Plot one band together with the outline of the selected subset to verify its geographic location.
plt.rcParams.update({'font.size': 14})
raster = band.ReadAsArray()
vmin = np.percentile(raster.flatten(), 5)
vmax = np.percentile(raster.flatten(), 95)
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.imshow(raster, cmap='gray', vmin=vmin, vmax=vmax)
# plot the subset as rectangle
_ = ax.add_patch(patches.Rectangle((subset[0], subset[1]), subset[2], subset[3], fill=False, edgecolor='red'))
plt.savefig('AOI.png', dpi=300, transparent='false')

In [ ]:
raster0 = band.ReadAsArray(*subset)
bandnbr = 0 # Needed for updates
rasterstack = img.ReadAsArray(*subset)

In [ ]:
img = None

In [ ]:
caldB = 20*np.log10(rasterstack) - 83

In [ ]:
calPwr = np.power(10., caldB/10.)

In [ ]:
os.chdir(path)
product_path = 'plots_and_animations'
new_directory(product_path)
if path_exists(product_path) and os.getcwd() != f"{path}/{product_path}":
    os.chdir(product_path)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
%%capture 
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
ax.axis('off')
vmin = np.percentile(caldB.flatten(), 5)
vmax = np.percentile(caldB.flatten(), 95)
r0dB = 20*np.log10(raster0) - 83
im = ax.imshow(r0dB,cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title("{}".format(tindex[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex[i].date()))
    im.set_data(caldB[i])

# Interval is given in milliseconds
ani = an.FuncAnimation(fig, animate, frames=rasterstack.shape[0], interval=400)

In [ ]:
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
ani.save('NepalTimeSeriesAnimation.gif', writer='pillow', fps=2)

## Plot Mean

In [ ]:
rs_means_pwr = np.mean(calPwr,axis=(1, 2))

In [ ]:
rs_means_dB = 10.*np.log10(rs_means_pwr)
rs_means_pwr.shape

In [ ]:
# 3. Now let's plot the time series of means
fig = plt.figure(figsize=(16, 5))
ax1 = fig.add_subplot(111)
ax1.plot(tindex, rs_means_pwr)
plt.rcParams.update({'font.size': 14})
ax1.set_xlabel('Date')
ax1.set_ylabel('$\overline{\gamma^o}$ [power]')
ax1.grid(axis='x')

ax2 = ax1.twinx()
plt.rcParams.update({'font.size': 14})
ax2.plot(tindex, rs_means_dB, color='red')
ax2.set_ylabel('$\overline{\gamma^o}$ [dB]')
fig.legend(['power', 'dB'], loc=1)
plt.title('Time series profile of average band backscatter $\gamma^o$ ')
plt.savefig('time_series_means', dpi=72, transparent='true')

In [ ]:
%%capture 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5), gridspec_kw={'width_ratios':[1, 3]})
plt.rcParams.update({'font.size': 14})
vmin = np.percentile(rasterstack.flatten(), 5)
vmax = np.percentile(rasterstack.flatten(), 95)
im = ax1.imshow(raster0, cmap='gray', vmin=vmin, vmax=vmax)
ax1.set_title("{}".format(tindex[0].date()))
ax1.set_axis_off()

ax2.axis([tindex[0].date(), tindex[-1].date(), rs_means_dB.min(), rs_means_dB.max()])
plt.rcParams.update({'font.size': 14})
ax2.set_ylabel('$\overline{\gamma^o}$ [dB]')
ax2.set_xlabel('Date')
ax2.set_ylim((-10, -5))
ax2.grid()
l, = ax2.plot([], [])
plt.tight_layout()

def animate(i):
    ax1.set_title("{}".format(tindex[i].date()))
    im.set_data(rasterstack[i])
    ax2.set_title("{}".format(tindex[i].date()))
    l.set_data(tindex[:(i+1)], rs_means_dB[:(i+1)])

# Interval is given in milliseconds
ani = an.FuncAnimation(fig, animate, frames=rasterstack.shape[0], interval=400)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
ani.save('NepalTSAnimation_means.gif', writer='pillow', fps=2)